## Install and Import Dependencies

In [ ]:
import cv2
import google.generativeai as genai
import mediapipe as mp
import numpy as np
import os
import pyttsx3
import random
from sklearn.metrics import confusion_matrix, classification_report
import time
import threading
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
engine = pyttsx3.init()
genai.configure(api_key="REPLACE_WITH_YOUR_KEY")

## Calculate Angle

In [6]:
def calculate_angle(a,b,c):
    a = np.array(a) # Start
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle > 180.0:
        angle = 360-angle
        
    return angle

## Speech to Text and Feedback

In [7]:
def speak_text(text):
    engine.say(text)
    engine.runAndWait()

In [8]:
def get_feedback(exercise, reps, time):
    prompt = f"Stroke rehabilitation. I did {exercise} exercise: {reps} reps in {time:.2f} seconds. Provide short feedback."
    model = genai.GenerativeModel("gemini-1.5-flash")
    response = model.generate_content(prompt)
    print("Prompt: " + prompt)
    print("Feedback: " + response.text)
    return response.text

## 1. Arm Raise (Upper Body)

In [ ]:
# Webcam input
cap = cv2.VideoCapture(0)

# Counter, stage and time variables
counter = 0
stage = "down"
start_time = None
end_time = None

# Intro instructions
speak_text("This is the arm raise exercise tracker.")
time.sleep(1)
speak_text("To perform the arm raise exercise, stand straight, keep your arms at your sides, and then raise both arms above your head, keeping your elbows straight.")

# Mediapipe pose detection
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Error: Unable to read from the camera.")
            break
        
        # Convert frame to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)
        
        # Convert frame back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get required landmarks
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                   landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_heel = landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].y
            right_heel = landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].y
            nose = landmarks[mp_pose.PoseLandmark.NOSE.value].y
            left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y
            right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y
            
            # Calculate angle at shoulder
            angle = calculate_angle(elbow, shoulder, hip)
            
            # Display angle on screen
            cv2.putText(image, str(int(angle)), 
                        tuple(np.multiply(shoulder, [640, 480]).astype(int)), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Arm raise logic
            if angle > 160 and stage == "down" and left_heel < 1 and right_heel < 1 and nose > 0 and left_wrist - 0.05 <= right_wrist <= left_wrist + 0.05:
                stage = "up"
                if counter == 0:
                    start_time = time.time() 
                counter += 1
                
            if angle < 80 and stage == "up" and left_heel < 1 and right_heel < 1 and nose > 0 and left_wrist - 0.05 <= right_wrist <= left_wrist + 0.05:
                stage = "down"
                end_time = time.time()
                threading.Thread(target=speak_text, args=(str(counter),)).start()
                print(f"Reps: {counter}")
        
        except Exception as e:
            pass
        
        # Display counter and stage
        cv2.rectangle(image, (0, 0), (275, 75), (245, 117, 16), -1)
        cv2.putText(image, str(counter) + " " + stage, 
                    (10, 60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Draw landmarks
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))
        
        # Show the feed
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [ ]:
if start_time and end_time:
    total_time = end_time - start_time
    speak_text(get_feedback("Arm Raise (Upper Body)", counter, total_time))
else:
    speak_text(get_feedback("Arm Raise (Upper Body)", counter, 0))

In [16]:
# Paths for video folders
correct_form_path = "Testing Videos/Arm Raise Correct"
incorrect_form_path = "Testing Videos/Arm Raise Incorrect"


# Metrics for confusion matrix
y_true = []  # Actual labels
y_pred = []  # Predicted labels

def process_video(video_path):
    # Counter and stage variables
    global flag, stage
    flag = False
    stage = "down"

    # Load video
    cap = cv2.VideoCapture(video_path)

    # Mediapipe pose detection
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break  # End of video

            # Convert frame to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = pose.process(image)

            # Convert frame back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            try:
                landmarks = results.pose_landmarks.landmark

                # Get required landmarks
                shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                       landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                left_heel = landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].y
                right_heel = landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].y
                nose = landmarks[mp_pose.PoseLandmark.NOSE.value].y
                left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y
                right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y

                # Calculate angle at shoulder
                angle = calculate_angle(elbow, shoulder, hip)

                # Arm raise logic
                if angle > 160 and stage == "down" and left_wrist - 0.05 <= right_wrist <= left_wrist + 0.05:
                    stage = "up"
                    
                if angle < 80 and stage == "up" and left_wrist - 0.05 <= right_wrist <= left_wrist + 0.05:
                    stage = "down"
                    flag = True

            except Exception as e:
                pass

        cap.release()
    
    return flag

# Process correct form videos
for video_file in os.listdir(correct_form_path):
    video_path = os.path.join(correct_form_path, video_file)
    reps = process_video(video_path)
    y_true.append(1)  # Actual label: Positive (Correct Form)
    y_pred.append(1 if flag else 0)  # Predicted label: Positive if reps detected

# Process incorrect form videos
for video_file in os.listdir(incorrect_form_path):
    video_path = os.path.join(incorrect_form_path, video_file)
    reps = process_video(video_path)
    y_true.append(0)  # Actual label: Negative (Incorrect Form)
    y_pred.append(1 if flag else 0)  # Predicted label: Positive if reps detected

print(y_true)
print(y_pred)

from sklearn.metrics import confusion_matrix, classification_report

# # Generate Confusion Matrix and Metrics
cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(cm)
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=["Incorrect Form", "Correct Form"],digits = 6))

c:\Users\shaur\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Confusion Matrix:
[[52  3]
 [ 8 43]]

Classification Report:
                precision    recall  f1-score   support

Incorrect Form   0.866667  0.945455  0.904348        55
  Correct Form   0.934783  0.843137  0.886598        51

      accuracy                       0.896226       106
     macro avg   0.900725  0.894296  0.895473       106
  weighted avg   0.

## 2. Knee Extension (Lower Body)

In [ ]:
# Webcam input
cap = cv2.VideoCapture(0)

# Counter, stage and time variables
counter = 0
stage = "down"
start_time = None
end_time = None

# Intro instructions
speak_text("This is the knee extension exercise tracker.")
time.sleep(1)
speak_text("To perform the knee extension exercise, sit on a chair, keep your feet flat on the ground, and then lift one leg, extending the knee fully while keeping the other leg bent.")

# Mediapipe pose detection
mp_pose = mp.solutions.pose
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        # Convert frame to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)

        # Convert frame back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark

            # Get required landmarks
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].z,
                   landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].z,
                    landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].z,
                      landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            
            hip_height = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y
            knee_height = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y
            left_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y
            right_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y

            left_heel = landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].y
            right_heel = landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].y
            nose = landmarks[mp_pose.PoseLandmark.NOSE.value].y
            
            
            kneeDisplay = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                    landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]

            # Calculate angle at knee
            angle = calculate_angle(hip, knee, ankle)

            # Display angle on screen
            cv2.putText(image, str(int(angle)), 
                        tuple(np.multiply(kneeDisplay, [640, 480]).astype(int)), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

            # Knee extension logic
            if angle > 170 and stage == "down" and knee_height - 0.15 <= hip_height <= knee_height + 0.15 and left_ankle - 0.1 <= right_ankle <= left_ankle + 0.1 and left_heel < 1 and right_heel < 1 and nose > 0:
                stage = "up"
                if counter == 0:
                    start_time = time.time() 
                counter += 1
            if angle < 100 and stage == "up" and knee_height - 0.15 <= hip_height <= knee_height + 0.15 and left_ankle - 0.1 <= right_ankle <= left_ankle + 0.1 and left_heel < 1 and right_heel < 1 and nose > 0:
                stage = "down"
                end_time = time.time()
                threading.Thread(target=speak_text, args=(str(counter),)).start()
                print(f"Reps: {counter}")

        except Exception as e:
            print(e)
            pass

        # Display counter and stage
        cv2.rectangle(image, (0, 0), (275, 75), (245, 117, 16), -1)
        cv2.putText(image, str(counter) + " " + stage,
                    (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

        # Draw landmarks
        mp_drawing = mp.solutions.drawing_utils
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

        # Show the feed
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [ ]:
if start_time and end_time:
    total_time = end_time - start_time
    speak_text(get_feedback("Knee Extension (Lower Body)", counter, total_time))
else:
    speak_text(get_feedback("Knee Extension (Lower Body)", counter, 0))

In [10]:
# Paths for video folders
correct_form_path = "Testing Videos/Knee Extension Correct"
incorrect_form_path = "Testing Videos/Knee Extension Incorrect"

# Metrics for confusion matrix
y_true = []  # Actual labels
y_pred = []  # Predicted labels

def process_video(video_path):
    # Flag and stage variables
    global flag, stage
    flag = False
    stage = "down"

    # Load video
    cap = cv2.VideoCapture(video_path)

    # Mediapipe pose detection
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break  # End of video

            # Convert frame to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = pose.process(image)

            # Convert frame back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            try:
                landmarks = results.pose_landmarks.landmark

                # Get required landmarks
                hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].z,
                   landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].z,
                        landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].z,
                        landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                
                hip_height = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y
                knee_height = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y
                left_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y
                right_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y

                # Calculate angle at knee
                angle = calculate_angle(hip, knee, ankle)

                # Knee extension logic
                if angle > 170 and stage == "down" and knee_height - 0.15 <= hip_height <= knee_height + 0.15 and left_ankle - 0.1 <= right_ankle <= left_ankle + 0.1:
                    stage = "up"
                if angle < 100 and stage == "up" and knee_height - 0.15 <= hip_height <= knee_height + 0.15 and left_ankle - 0.1 <= right_ankle <= left_ankle + 0.1:
                    stage = "down"
                    flag = True

            except Exception as e:
                print(e)
                pass

        cap.release()
    
    return flag

# Process correct form videos
for video_file in os.listdir(correct_form_path):
    video_path = os.path.join(correct_form_path, video_file)
    reps = process_video(video_path)
    y_true.append(1)  # Actual label: Positive (Correct Form)
    y_pred.append(1 if flag else 0)  # Predicted label: Positive if reps detected

# Process incorrect form videos
for video_file in os.listdir(incorrect_form_path):
    video_path = os.path.join(incorrect_form_path, video_file)
    reps = process_video(video_path)
    y_true.append(0)  # Actual label: Negative (Incorrect Form)
    y_pred.append(1 if flag else 0)  # Predicted label: Positive if reps detected

print(y_true)
print(y_pred)

from sklearn.metrics import confusion_matrix, classification_report

# Generate Confusion Matrix and Metrics
cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(cm)
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=["Incorrect Form", "Correct Form"], digits = 6))

c:\Users\shaur\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute

## 3. Sit To Stand (Full Body)

In [ ]:
# Webcam input
cap = cv2.VideoCapture(0)

# Counter, stage, and time variables
counter = 0
stage = "sit"
start_time = None
end_time = None

# Intro instructions
speak_text("This is the sit-to-stand exercise tracker.")
time.sleep(1)
speak_text("To perform the sit-to-stand exercise, sit on a chair, then rise to a standing position, and return to sitting.")

# Mediapipe pose detection
mp_pose = mp.solutions.pose
with mp_pose.Pose(min_detection_confidence=0.7, min_tracking_confidence=0.7) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Error: Unable to read from the camera.")
            break

        # Convert frame to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)

        # Convert frame back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            # Extract pose landmarks
            landmarks = results.pose_landmarks.landmark

            # Get required landmark heights
            hip_height = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y
            knee_height = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y
            ankle_height = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y
            right_ankle_height = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y
            left_heel = landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].y
            right_heel = landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].y
            nose = landmarks[mp_pose.PoseLandmark.NOSE.value].y

            hip_display = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                           landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            
            # Calculate the ratio between hip to ankle and hip to knee heights
            height_ratio = abs(hip_height - ankle_height) / abs(hip_height - knee_height)

            cv2.putText(image, f"Height Ratio: {height_ratio:.2f}",
                        tuple(np.multiply(hip_display, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

            # Sit-to-stand logic
            if height_ratio < 2  and stage == "sit" and left_heel < 1 and right_heel < 1 and nose > 0 and ankle_height - 0.05 <= right_ankle_height <= ankle_height + 0.05 and hip_height <= knee_height - 0.08:
                stage = "stand"
                if counter == 0:
                    start_time = time.time() 
                counter += 1

            if height_ratio > 2.25 and stage == "stand" and left_heel < 1 and right_heel < 1 and nose > 0 and ankle_height - 0.05 <= right_ankle_height <= ankle_height + 0.05 and hip_height <= knee_height - 0.08:
                stage = "sit"
                end_time = time.time()
                threading.Thread(target=speak_text, args=(str(counter),)).start()
                print(f"Reps: {counter}")

        except Exception as e:
            pass

        # Display counter and stage
        cv2.rectangle(image, (0, 0), (275, 75), (245, 117, 16), -1)
        cv2.putText(image, f"{counter} {stage}",
                    (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

        # Draw landmarks
        mp_drawing = mp.solutions.drawing_utils
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

        # Show the feed
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [ ]:
if start_time and end_time:
    total_time = end_time - start_time
    speak_text(get_feedback("Sit To Stand (Full Body)", counter, total_time))
else:
    speak_text(get_feedback("Sit To Stand (Full Body)", counter, 0))

In [11]:
# Paths for video folders
correct_form_path = "Testing Videos/Sit To Stand Correct"
incorrect_form_path = "Testing Videos/Sit To Stand Incorrect"

# Metrics for confusion matrix
y_true = []  # Actual labels
y_pred = []  # Predicted labels

def process_video(video_path):
    # Flag and stage variables
    global flag, stage
    flag = False
    stage = "sit"

    # Load video
    cap = cv2.VideoCapture(video_path)

    # Mediapipe pose detection
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break  # End of video

            # Convert frame to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = pose.process(image)

            # Convert frame back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            try:
                landmarks = results.pose_landmarks.landmark

                # Get required landmarks
                hip_height = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y
                knee_height = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y
                ankle_height = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y
                right_ankle_height = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y
                left_heel = landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].y
                right_heel = landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].y
                nose = landmarks[mp_pose.PoseLandmark.NOSE.value].y

                hip_display = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                
                # Calculate the ratio between hip to ankle and hip to knee heights
                height_ratio = abs(hip_height - ankle_height) / abs(hip_height - knee_height)

                cv2.putText(image, f"Height Ratio: {height_ratio:.2f}",
                            tuple(np.multiply(hip_display, [640, 480]).astype(int)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

                # Sit-to-stand logic
                if height_ratio < 2.00  and stage == "sit" and left_heel < 1 and right_heel < 1 and nose > 0 and ankle_height - 0.05 <= right_ankle_height <= ankle_height + 0.05 and hip_height <= knee_height - 0.08:
                    stage = "stand"
                    
                if height_ratio > 2.25 and stage == "stand" and left_heel < 1 and right_heel < 1 and nose > 0 and ankle_height - 0.05 <= right_ankle_height <= ankle_height + 0.05 and hip_height <= knee_height - 0.08:
                    stage = "sit"
                    flag = True

            except Exception as e:
                pass

        cap.release()
    
    return flag

# Process correct form videos
for video_file in os.listdir(correct_form_path):
    video_path = os.path.join(correct_form_path, video_file)
    reps = process_video(video_path)
    y_true.append(1)  # Actual label: Positive (Correct Form)
    y_pred.append(1 if flag else 0)  # Predicted label: Positive if reps detected

# Process incorrect form videos
for video_file in os.listdir(incorrect_form_path):
    video_path = os.path.join(incorrect_form_path, video_file)
    reps = process_video(video_path)
    y_true.append(0)  # Actual label: Negative (Incorrect Form)
    y_pred.append(1 if flag else 0)  # Predicted label: Positive if reps detected

print(y_true)
print(y_pred)

from sklearn.metrics import confusion_matrix, classification_report

# # Generate Confusion Matrix and Metrics
cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(cm)
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=["Incorrect Form", "Correct Form"], digits = 6))

c:\Users\shaur\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0]
Confusion Matrix:
[[49 13]
 [ 2 77]]

Classification Report:
                precision    recall  f1-score   support

Incorrect Form   0.960784  0.79032

## 4. Tracing Patterns (Balance)

In [ ]:
# Webcam input
cap = cv2.VideoCapture(0)

# Counter, target variables, and timing
counter = 0
num_targets = 15  # Total number of targets
target_position = [random.randint(100, 500), random.randint(100, 400)]  # Initial target position
target_radius = 15  # Radius of the target
move_interval = 0.5  # Time in seconds between movements
last_move_time = time.time()

# Check if a landmark is near the target
def is_near_target(point, target, threshold=30):
    distance = ((point[0] - target[0])**2 + (point[1] - target[1])**2)**0.5
    return distance < threshold

# Generate a random position for the target within the screen bounds
def generate_random_position():
    return [random.randint(100, 500), random.randint(100, 400)]

# Intro instructions
speak_text("This is the balance and coordination exercise tracker.")
time.sleep(1)
speak_text("Trace the moving points on the screen using your hands or legs.")

# Mediapipe pose detection
mp_pose = mp.solutions.pose
with mp_pose.Pose(min_detection_confidence=0.7, min_tracking_confidence=0.7) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Error: Unable to read from the camera.")
            break

        # Mirror the frame (flip horizontally)
        frame = cv2.flip(frame, 1)

        # Convert frame to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)

        # Convert frame back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            # Extract pose landmarks
            landmarks = results.pose_landmarks.landmark

            # Get positions for hand and foot
            left_hand = [int(landmarks[mp_pose.PoseLandmark.LEFT_INDEX.value].x * 640),
                         int(landmarks[mp_pose.PoseLandmark.LEFT_INDEX.value].y * 480)]
            left_foot = [int(landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x * 640),
                         int(landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y * 480)]
            right_hand = [int(landmarks[mp_pose.PoseLandmark.RIGHT_INDEX.value].x * 640),
                          int(landmarks[mp_pose.PoseLandmark.RIGHT_INDEX.value].y * 480)]
            right_foot = [int(landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x * 640),
                          int(landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y * 480)]

            # Draw the current target point
            cv2.circle(image, tuple(target_position), target_radius, (0, 0, 255), -1)

            # Check if a hand or foot is near the current target
            if (is_near_target(left_hand, target_position) or
                    is_near_target(left_foot, target_position) or
                    is_near_target(right_hand, target_position) or
                    is_near_target(right_foot, target_position)):
                counter += 1
                threading.Thread(target=speak_text, args=(f"Target {counter} touched!",)).start()
                target_position = generate_random_position()  # Move to the next random position

                # Check if the required number of targets has been hit
                if counter >= num_targets:
                    speak_text("Exercise complete!")
                    break

            # Move the target periodically
            if time.time() - last_move_time > move_interval:
                target_position[0] += random.choice([-20, 20])
                target_position[1] += random.choice([-20, 20])
                # Keep target within screen bounds
                target_position[0] = max(50, min(target_position[0], 590))
                target_position[1] = max(50, min(target_position[1], 430))
                last_move_time = time.time()

        except Exception as e:
            print(e)
            pass

        # Display counter
        cv2.rectangle(image, (0, 0), (350, 75), (245, 117, 16), -1)
        cv2.putText(image, f"Targets Hit: {counter} / {num_targets}",
                    (10, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        # Draw landmarks
        mp_drawing = mp.solutions.drawing_utils
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

        # Show the feed
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()